In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
from keras.optimizers import RMSprop, Adam
from keras.models import load_model
import numpy as np
import random
import os
import time
import json

from src.read_data import read_trump_speeches
from src.utils import print_tensorflow_devices
from src.data_generator import DataGenerator
from src.examples_generator_callback import ExamplesGeneratorCallback

Using TensorFlow backend.


In [2]:
# Parameters
seq_len = 21 # includes next word.
batch_size = 128
epochs = 50

In [3]:
speeches = read_trump_speeches('data/speeches.txt')
words = np.unique(speeches)
word_index = dict((c, i) for i, c in enumerate(words))
index_word = dict((i, c) for i, c in enumerate(words))
n_words = len(words)

# Index the speeches, then create sentences of length 'seq_len' that we can train the RNN on.
speeches_indexed = [word_index[x] for x in speeches]
sentence_ranges = [range(i, i + seq_len) for i in range(0, len(speeches) - seq_len)]
sentences = [[speeches[y] for y in x] for x in sentence_ranges]
sentences_indexed = [[speeches_indexed[y] for y in x] for x in sentence_ranges]

# Train-test split
random.seed(12345)
random.shuffle(sentences_indexed)
train_split = int(0.95 * len(sentences_indexed))
sentences_indexed_train = sentences_indexed[:train_split]
sentences_indexed_test = sentences_indexed[train_split:]
print('Train: ' + str(len(sentences_indexed_train)))
print('Test: ' + str(len(sentences_indexed_test)))

Train: 176350
Test: 9282


In [4]:
def get_model(version):
    print('Build model...')
    model = Sequential()
    model.add(Bidirectional(LSTM(200, activation="relu", return_sequences=True), input_shape=(seq_len - 1, n_words)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(200, activation="relu")))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_words, activation='softmax'))
    return model


In [5]:
model_id = time.strftime("%Y%m%d-%H%M%S")
log_dir = 'log/' + model_id
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

model_to_be_loaded = None
if model_to_be_loaded is not None:
    model = load_model('models/' + model_to_be_loaded)
else:
    model = get_model()

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

with open(log_dir + '/summary.txt', 'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

with open(log_dir + '/configuration.txt', 'w') as fout:
    json.dump(model.get_config(), fout)

print_callback = ExamplesGeneratorCallback(sentences_indexed_test, index_word, log_dir + '/examples.txt', seq_len,
                                           n_words)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
csv_logger = CSVLogger(log_dir + '/loss_log.csv', append=True, separator=';')
callbacks_list = [print_callback, early_stopping, csv_logger]

history = model.fit_generator(DataGenerator(sentences_indexed_train, seq_len, n_words, batch_size),
                              steps_per_epoch=int(len(sentences_indexed_train) / batch_size) + 1,
                              epochs=epochs,
                              callbacks=callbacks_list,
                              validation_data=DataGenerator(sentences_indexed_test, seq_len, n_words, batch_size),
                              validation_steps=int(len(sentences_indexed_test) / batch_size) + 1)

model.save(log_dir + '/' + time.strftime("%Y%m%d-%H%M%S") + '.h5')


TypeError: get_model() missing 1 required positional argument: 'version'